<a href="https://colab.research.google.com/github/moseskim/bert_nlp/blob/main/section_5/01_news_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 일본어 문장 분류

일본어 데이터셋으로 BERT 모델을 파인 튜닝하고, 뉴스를 분류합니다.

## 라이브러리 설치

라이브러리인 Transformers, 및 `nlp`를 설치합니다.

In [ ]:
!pip install transformers
!pip install nlp
!pip install datasets
!pip install fugashi
!pip install ipadic
!pip install sentencepiece
!pip install accelerate

## Google 드라이브와의 연동

다음 코드를 실행하고 인증 코드를 사용해 Google 드라이브를 마운트합니다.

In [ ]:
from google.colab import drive
drive.mount("/content/drive/")

## 데이터셋 로딩

Google 드라이브에 저장되어 있는 뉴스 데이터셋을 로딩합니다.

In [ ]:
import glob  # 파일 취득에 사용
import os

path = "/content/drive/My Drive/bert_nlp/section_5/text/"  # 풀더 위치 지정

dir_files = os.listdir(path=path)
dirs = [f for f in dir_files if os.path.isdir(os.path.join(path, f))]  # 디렉터리 목록

text_label_data = []  # 문장과 라벨 셋
dir_count = 0  # 디렉터리 수 카운트
file_count= 0  # 파일 수 카운트

for i in range(len(dirs)):
    dir = dirs[i]
    files = glob.glob(path + dir + "/*.txt")  # 파일 목록
    dir_count += 1

    for file in files:
        if os.path.basename(file) == "LICENSE.txt":
            continue

        with open(file, "r") as f:
            text = f.readlines()[3:]
            text = "".join(text)
            text = text.translate(str.maketrans({"\n":"", "\t":"", "\r":"", "\u3000":""}))
            text_label_data.append([text, i])

        file_count += 1
        print("\rfiles: " + str(file_count) + "dirs: " + str(dir_count), end="")

## 데이터 저장

데이터를 훈련 데이터와 테스트 데이트로 분할하고, `csv` 파일로 Google Drive에 저장합니다.

In [ ]:
import csv
from sklearn.model_selection import train_test_split

news_train, news_test =  train_test_split(text_label_data, shuffle=True)  # 훈련용과 테스트용으로 분할
news_path = "/content/drive/My Drive/bert_nlp/section_5/"

with open(news_path+"news_train.csv", "w") as f:
    writer = csv.writer(f)
    writer.writerows(news_train)

with open(news_path+"news_test.csv", "w") as f:
    writer = csv.writer(f)
    writer.writerows(news_test)

## 모델과 Tokenizer 로딩

일본어 사전 학습 완료 모델 및 이와 관련된 Tokenizer를 로딩합니다.

In [ ]:
from transformers import BertForSequenceClassification, BertJapaneseTokenizer

sc_model = BertForSequenceClassification.from_pretrained("cl-tohoku/bert-base-japanese-whole-word-masking", num_labels=9)
sc_model.cuda()
tokenizer = BertJapaneseTokenizer.from_pretrained("cl-tohoku/bert-base-japanese-whole-word-masking")

## 데이터셋 로딩

저장된 뉴스 데이터를 로딩합니다.  

In [ ]:
from datasets import load_dataset

def tokenize(batch):
    return tokenizer(batch["text"], padding=True, truncation=True, max_length=128)

news_path = "/content/drive/My Drive/bert_nlp/section_5/"

train_data = load_dataset("csv", data_files=news_path+"news_train.csv", column_names=["text", "label"], split="train")
train_data = train_data.map(tokenize, batched=True, batch_size=len(train_data))
train_data.set_format("torch", columns=["input_ids", "label"])

test_data = load_dataset("csv", data_files=news_path+"news_test.csv", column_names=["text", "label"], split="train")
test_data = test_data.map(tokenize, batched=True, batch_size=len(test_data))
test_data.set_format("torch", columns=["input_ids", "label"])

## 평가용 함수

`sklearn.metrics`를 사용해 모델을 평가하기 위한 함수를 정의합니다.

In [ ]:
from sklearn.metrics import accuracy_score

def compute_metrics(result):
    labels = result.label_ids
    preds = result.predictions.argmax(-1)
    acc = accuracy_score(labels, preds)
    return {
        "accuracy": acc,
    }

## Trainer 설정

`Trainer` 클래스 및 `TrainingArguments` 클래스를 사용해 훈련을 수행할 Trainer를 설정합니다.  
https://huggingface.co/transformers/main_classes/trainer.html   
https://huggingface.co/transformers/main_classes/trainer.html#trainingarguments  

In [ ]:
from transformers import Trainer, TrainingArguments

training_args = TrainingArguments(
    output_dir = "./results",
    num_train_epochs = 2,
    per_device_train_batch_size = 8,
    per_device_eval_batch_size = 32,
    warmup_steps = 500,  # 학습 계수가 0부터 이 단계 수로 상승
    weight_decay = 0.01,  # 가중치 감쇠율
    logging_dir = "./logs",
    evaluation_strategy = "steps"
)

trainer = Trainer(
    model = sc_model,
    args = training_args,
    compute_metrics = compute_metrics,
    train_dataset = train_data,
    eval_dataset = test_data,
)

## 모델 훈련

설정에 기반해 파인 튜닝을 수행합니다.

In [ ]:
trainer.train()

## 모델 평가

Trainer의 `evaluate()` 메서드를 사용해 모델을 평가합니다.

In [ ]:
trainer.evaluate()

## TensorBoard를 사용한 결과 표시

TensorBoard를 사용해 `logs` 폴더에 저장된 학슴 과정을 표시합니다.

In [ ]:
%load_ext tensorboard
%tensorboard --logdir logs

## 모델 저장

훈련 완료 모델을 저장합니다.

In [ ]:
news_path = "/content/drive/My Drive/bert_nlp/section_5/"

sc_model.save_pretrained(news_path)
tokenizer.save_pretrained(news_path)

## 모델 로딩

저장 완료 모델을 로딩합니다.  

In [ ]:
loaded_model = BertForSequenceClassification.from_pretrained(news_path)
loaded_model.cuda()
loaded_tokenizer = BertJapaneseTokenizer.from_pretrained(news_path)

## 일본어 뉴스 분류

로딩한 모델을 사용해 뉴스를 분류합니다.

In [ ]:
import glob  # ファイルの取得に使用
import os
import torch

category = "movie-enter"
sample_path = "/content/drive/My Drive/bert_nlp/section_5/text/"  # 폴더 위치 지정
files = glob.glob(sample_path + category + "/*.txt")  # 파일 목록
file = files[12]  # 임의의 뉴스

dir_files = os.listdir(path=sample_path)
dirs = [f for f in dir_files if os.path.isdir(os.path.join(sample_path, f))]  # 디렉터리 목록

with open(file, "r") as f:
    sample_text = f.readlines()[3:]
    sample_text = "".join(sample_text)
    sample_text = sample_text.translate(str.maketrans({"\n":"", "\t":"", "\r":"", "\u3000":""}))

print(sample_text)

max_length = 512
words = loaded_tokenizer.tokenize(sample_text)
word_ids = loaded_tokenizer.convert_tokens_to_ids(words)  # 단어를 인덱스로 변환
word_tensor = torch.tensor([word_ids[:max_length]])  # 텐서로 변환

x = word_tensor.cuda()  # GPU 대응
y = loaded_model(x)  # 예측
pred = y[0].argmax(-1)  # 최댓값의 인덱스
print("result:", dirs[pred])